In [7]:
import torch
from torch import nn, tensor, randn
import pandas
# import nltk # See https://www.nltk.org/data.html
from nltk import tokenize
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
# Translate data from original CSV file into strings

data = pandas.read_csv("data/glados-portal2.original.csv", sep="|", index_col=0)["d1"]
file = open("data/glados-portal2.csv", "w")
for string in data:
    file.write(string + "\n")

In [5]:
char_tokenize = lambda text: nltk.tokenize.simple.CharTokenizer().tokenize(text)
word_tokenize = lambda text: [t.lower() for t in tokenize.WordPunctTokenizer().tokenize(text)]

# T (Time) -- размер контекстного окна
get_x = lambda data, T, pos: data[pos : pos + T]
get_y = lambda data, T, pos: data[pos + 1 : pos + T + 1]
# get_x_y = lambda data, T, pos: data[pos : pos + T], data[pos + 1 : pos + T + 1]

# B -- batch size
def get_batch(data, B, T):
    positions = torch.randint(len(data) - T, (B,))
    batch_x = torch.stack([get_x(data, T, pos) for pos in positions])
    batch_y = torch.stack([get_y(data, T, pos) for pos in positions])

In [ ]:
with open("data/glados-portal2.csv", "r") as f:
    tokens = word_tokenize(f.read())

vocab = list(set(tokens))
C = len(vocab)

id_to_token = vocab
token_to_id = { t:i for i, t in enumerate(vocab)}

torch.manual_seed(0)

class DigramLM(nn.Module):
    def __init__(self, vocab):
        super.__init__()
        self.token_embedding_table = nn.Embedding(C, C)
    
    # def forward(self, input, targets):

# class LSTMLM(nn.Model):

# class TransformerLM(nn.Model):


# embedding = nn.Embedding(len(token_set), 2)
# display(torch.LongTensor([1]))
# print(len(token_set))
# embed = embedding(torch.LongTensor([1]))
# # X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
# # inputs = tensor()

# # ᵺ = 2
# input_size = len(token_set)
# hidden_size = 10
# # data_strings = file.readlines()
# layer_number = 2

# lstm = nn.LSTM(input_size, hidden_size, layer_number)

# print(lstm(randn(1, len(token_set))))

AttributeError: module 'torch.nn' has no attribute 'Model'

In [29]:

class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_size=128):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    
    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        return self.fc(out), hidden

In [30]:
with open("data/glados-portal2.csv", "r") as f:
    text = f.read()

tokens = word_tokenize(text.lower())
vocab =  sorted(list(set(tokens)))
vocab = ['<PAD>', '<UNK>'] + vocab
vocab_size = len(vocab)

token_to_id = {t:i for i,t in enumerate(vocab)}
id_to_token = {i:t for t,i in token_to_id.items()}

torch.manual_seed(0)


def tokens_to_text(token_ids, id_to_token):
    return " ".join([id_to_token.get(token_id, "<UNK>") for token_id in token_ids])

In [32]:

vocab_size = C  # Размер словаря
seq_len = 20       # Длина последовательности


model = SimpleLSTM(vocab_size)


input = torch.randint(0, vocab_size, (1, seq_len))

# Прямой проход
output, hidden = model(input)
print(f"Input shape: {input.shape}")
print(f"Output shape: {output.shape}") 

# Генерация текста
def generate(model, start_token, length=10, id_to_token=id_to_token):
    model.eval()
    tokens = [start_token]
    hidden = None
    
    for _ in range(length):
        input_tensor = torch.tensor([tokens[-1]]).unsqueeze(0)
        output, hidden = model(input_tensor, hidden)
        next_token = output.argmax(-1).item()
        tokens.append(next_token)
    
    return tokens_to_text(tokens, id_to_token)

# Пример генерации
start_token = token_to_id.get('drops', token_to_id['<UNK>']) 
generated = generate(model, start_token, length=15)
print("Generated text:", generated)


Input shape: torch.Size([1, 20])
Output shape: torch.Size([1, 20, 1970])
Generated text: drops b important disappointment survived and myself example around panels encouragement collection let fully showed harder
